In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA fraud_detection")


DataFrame[]

In [0]:
%sql
CREATE OR REPLACE VIEW gold_transactions_enriched
AS
SELECT
    transaction_id,
    step,
    transaction_type,
    amount,

    origin_account,
    destination_account,

    oldBalanceOrig,
    newBalanceOrig,
    oldBalanceDest,
    newBalanceDest,

    isUnauthorizedOverdraft,

    /* ---------- Derived Risk Features ---------- */

    CASE
        WHEN oldBalanceOrig < amount THEN 1
        ELSE 0
    END AS balance_mismatch_flag,

    CASE
        WHEN amount >= 200000 THEN 1
        ELSE 0
    END AS is_high_value_transaction,

    CASE
        WHEN transaction_type IN ('TRANSFER', 'CASH_OUT') THEN 1
        ELSE 0
    END AS is_risky_transaction_type,

    ingest_time
FROM silver_fact_transactions;


In [0]:
%sql
SELECT COUNT(*) FROM gold_transactions_enriched;


COUNT(*)
5442996


In [0]:
%sql
SELECT *
FROM gold_transactions_enriched
LIMIT 5;


transaction_id,step,transaction_type,amount,origin_account,destination_account,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,isUnauthorizedOverdraft,balance_mismatch_flag,is_high_value_transaction,is_risky_transaction_type,ingest_time
e87bd2ce-e892-4ff2-b559-1dff52a96e0b,1,CASH_IN,15504.98,C7252612212,M4639849776,183279.61,198784.59,0.0,0.0,0,0,0,0,2026-02-08T21:37:25.820Z
ddcec6e4-aeb5-4165-8a85-5e60304013aa,1,CASH_IN,83938.99,C7567192866,M0172269636,5578.3,89517.29,0.0,0.0,0,1,0,0,2026-02-08T21:37:25.820Z
e13f5635-788a-4617-8bf7-7388acefe360,1,CASH_IN,131805.11,C8879445092,M2384302511,29222.15,161027.26,0.0,0.0,0,1,0,0,2026-02-08T21:37:25.820Z
1402e4cf-9173-4b00-bdac-a6ee620a509b,1,CASH_IN,176931.04,C7110900402,M8592581184,52851.85,229782.89,0.0,0.0,0,1,0,0,2026-02-08T21:37:25.820Z
9b986898-70af-4d30-9f7e-c3b2da193cc9,1,CASH_OUT,268084.67,CC1229472380,M0141939366,268084.67,0.0,0.0,0.0,0,0,1,1,2026-02-08T21:37:25.820Z


In [0]:
%sql
DESCRIBE gold_transactions_enriched;


col_name,data_type,comment
transaction_id,string,null
step,int,null
transaction_type,string,null
amount,double,null
origin_account,string,null
destination_account,string,null
oldBalanceOrig,double,null
newBalanceOrig,double,null
oldBalanceDest,double,null
newBalanceDest,double,null


In [0]:
%sql show tables in fraud_detection;

database,tableName,isTemporary
fraud_detection,bronze_country_code,false
fraud_detection,bronze_customers,false
fraud_detection,bronze_fraud_report,false
fraud_detection,bronze_transactions,false
fraud_detection,gold_transactions_enriched,false
fraud_detection,silver_dim_accounts,false
fraud_detection,silver_dim_countries,false
fraud_detection,silver_dim_country,false
fraud_detection,silver_dim_customer,false
fraud_detection,silver_dim_customers,false


In [0]:
%sql
CREATE OR REPLACE VIEW gold_transactions_summary AS
SELECT
    COUNT(*)                                AS total_transactions,
    SUM(amount)                             AS total_amount,
    AVG(amount)                             AS avg_transaction_amount,
    SUM(CASE WHEN transaction_type = 'CASH_IN' THEN 1 ELSE 0 END) AS cash_in_count,
    SUM(CASE WHEN transaction_type = 'CASH_OUT' THEN 1 ELSE 0 END) AS cash_out_count,
    SUM(CASE WHEN transaction_type = 'TRANSFER' THEN 1 ELSE 0 END) AS transfer_count,
    SUM(isUnauthorizedOverdraft)            AS overdraft_events
FROM silver_fact_transactions;


In [0]:
%sql
SELECT * FROM gold_transactions_summary;


total_transactions,total_amount,avg_transaction_amount,cash_in_count,cash_out_count,transfer_count,overdraft_events
5442996,6.502657508176909E11,119468.34993406037,2124428,1103229,659954,12380


In [0]:
%sql
CREATE OR REPLACE VIEW gold_transactions_by_country AS
SELECT
    c.country_code,
    COUNT(*)            AS transaction_count,
    SUM(f.amount)       AS total_amount,
    AVG(f.amount)       AS avg_amount
FROM silver_fact_transactions f
JOIN silver_dim_country c
    ON f.origin_account IS NOT NULL
GROUP BY c.country_code;


In [0]:
%sql
SELECT * FROM gold_transactions_by_country
ORDER BY total_amount DESC
LIMIT 10;


country_code,transaction_count,total_amount,avg_amount
REU,5442996,6.502657508176909E11,119468.34993406037
BRA,5442996,6.502657508176909E11,119468.34993406037
ESP,5442996,6.502657508176909E11,119468.34993406037
QAT,5442996,6.502657508176909E11,119468.34993406037
ITA,5442996,6.502657508176909E11,119468.34993406037
RUS,5442996,6.502657508176909E11,119468.34993406037
PAN,5442996,6.502657508176909E11,119468.34993406037
KHM,5442996,6.502657508176909E11,119468.34993406037
PSE,5442996,6.502657508176909E11,119468.34993406037
CAN,5442996,6.502657508176909E11,119468.34993406037


In [0]:
%sql
CREATE OR REPLACE VIEW gold_cross_border_activity AS
SELECT
    origin.country_code      AS origin_country,
    dest.country_code        AS destination_country,
    COUNT(*)                 AS transaction_count,
    SUM(f.amount)            AS total_amount,
    AVG(f.amount)            AS avg_amount
FROM silver_fact_transactions f
JOIN silver_dim_country origin
    ON f.origin_account IS NOT NULL
JOIN silver_dim_country dest
    ON f.destination_account IS NOT NULL
WHERE origin.country_code <> dest.country_code
GROUP BY origin.country_code, dest.country_code;


In [0]:
%sql
SELECT *
FROM gold_cross_border_activity
ORDER BY total_amount DESC
LIMIT 10;


origin_country,destination_country,transaction_count,total_amount,avg_amount
ITA,QAT,5442996,6.502657508176909E11,119468.34993406037
NGA,KHM,5442996,6.502657508176909E11,119468.34993406037
TUR,PSE,5442996,6.502657508176909E11,119468.34993406037
REU,CAN,5442996,6.502657508176909E11,119468.34993406037
ESP,CAN,5442996,6.502657508176909E11,119468.34993406037
KHM,QAT,5442996,6.502657508176909E11,119468.34993406037
CAN,PER,5442996,6.502657508176909E11,119468.34993406037
BRA,KHM,5442996,6.502657508176909E11,119468.34993406037
ESP,QAT,5442996,6.502657508176909E11,119468.34993406037
RUS,ESP,5442996,6.502657508176909E11,119468.34993406037


In [0]:
%sql
CREATE OR REPLACE VIEW gold_risk_indicators AS
SELECT
    transaction_id,
    step,
    transaction_type,
    amount,
    oldBalanceOrig,
    newBalanceOrig,
    isUnauthorizedOverdraft,

    CASE
        WHEN amount > 200000 THEN 'HIGH_AMOUNT'
        WHEN isUnauthorizedOverdraft = 1 THEN 'OVERDRAFT'
        WHEN oldBalanceOrig = 0 AND newBalanceOrig > 0 THEN 'SUSPICIOUS_TOPUP'
        ELSE 'NORMAL'
    END AS risk_flag
FROM silver_fact_transactions;


In [0]:
%sql
SELECT risk_flag, COUNT(*)
FROM gold_risk_indicators
GROUP BY risk_flag;


risk_flag,COUNT(*)
NORMAL,4352481
SUSPICIOUS_TOPUP,526
OVERDRAFT,4454
HIGH_AMOUNT,1085535


In [0]:
%sql
CREATE OR REPLACE VIEW workspace.fraud_detection.gold_transaction_kpis AS
SELECT
    COUNT(*)                            AS total_transactions,
    SUM(amount)                         AS total_amount,
    AVG(amount)                         AS avg_transaction_amount,
    COUNT(DISTINCT origin_account)      AS unique_origin_accounts,
    COUNT(DISTINCT destination_account) AS unique_destination_accounts
FROM workspace.fraud_detection.silver_fact_transactions;


In [0]:
%sql
SELECT * FROM workspace.fraud_detection.gold_transaction_kpis;


total_transactions,total_amount,avg_transaction_amount,unique_origin_accounts,unique_destination_accounts
5442996,6.502657508176909E11,119468.34993406037,81162,88774


In [0]:
%sql
DESCRIBE silver_dim_accounts;
DESCRIBE silver_dim_countries;


col_name,data_type,comment
country_code,string,null


In [0]:
%sql
DESCRIBE silver_dim_accounts;


col_name,data_type,comment
account_id,string,null


In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA fraud_detection;


In [0]:
%sql
SHOW TABLES LIKE 'silver_dim_%';


database,tableName,isTemporary


In [0]:
%sql
SHOW TABLES IN workspace.fraud_detection;


database,tableName,isTemporary
fraud_detection,bronze_country_code,false
fraud_detection,bronze_customers,false
fraud_detection,bronze_fraud_report,false
fraud_detection,bronze_transactions,false
fraud_detection,gold_cross_border_activity,false
fraud_detection,gold_risk_indicators,false
fraud_detection,gold_transaction_kpis,false
fraud_detection,gold_transactions_by_country,false
fraud_detection,gold_transactions_enriched,false
fraud_detection,gold_transactions_summary,false


In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA fraud_detection;


In [0]:
%sql
SELECT * FROM gold_transaction_kpis;


total_transactions,total_amount,avg_transaction_amount,unique_origin_accounts,unique_destination_accounts
5442996,6.502657508176909E11,119468.34993406037,81162,88774


In [0]:
%sql
SELECT
  date_trunc('day', ingest_time) AS txn_day,
  COUNT(*) AS transaction_count,
  SUM(amount) AS total_amount
FROM gold_transactions_enriched
GROUP BY 1
ORDER BY 1;


txn_day,transaction_count,total_amount
2026-02-08T00:00:00.000Z,5442996,6.502657508176909E11


In [0]:
%sql
SELECT *
FROM gold_cross_border_activity
ORDER BY transaction_count DESC;


origin_country,destination_country,transaction_count,total_amount,avg_amount
PER,ESP,5442996,6.502657508176909E11,119468.34993406037
NGA,CAN,5442996,6.502657508176909E11,119468.34993406037
PSE,ITA,5442996,6.502657508176909E11,119468.34993406037
QAT,FRA,5442996,6.502657508176909E11,119468.34993406037
QAT,RUS,5442996,6.502657508176909E11,119468.34993406037
PAN,CAN,5442996,6.502657508176909E11,119468.34993406037
TUR,NGA,5442996,6.502657508176909E11,119468.34993406037
RUS,TUR,5442996,6.502657508176909E11,119468.34993406037
REU,NGA,5442996,6.502657508176909E11,119468.34993406037
CAN,PER,5442996,6.502657508176909E11,119468.34993406037


In [0]:
%sql
DESCRIBE gold_risk_indicators;


col_name,data_type,comment
transaction_id,string,null
step,int,null
transaction_type,string,null
amount,double,null
oldBalanceOrig,double,null
newBalanceOrig,double,null
isUnauthorizedOverdraft,int,null
risk_flag,string,null


In [0]:
%sql
SELECT *
FROM gold_risk_indicators
ORDER BY risk_flag DESC, amount DESC;

transaction_id,step,transaction_type,amount,oldBalanceOrig,newBalanceOrig,isUnauthorizedOverdraft,risk_flag
2ceb1b07-94fc-427a-a25c-4ced9e10c6aa,8,CASH_IN,198637.36,0.0,198637.36,0,SUSPICIOUS_TOPUP
458a3a2c-5720-4bcb-906b-ff2c8a4cd0d0,3,CASH_IN,198605.66,0.0,198605.66,0,SUSPICIOUS_TOPUP
d162e258-91f2-4593-a55e-05273cc6f462,132,CASH_IN,198530.92,0.0,198530.92,0,SUSPICIOUS_TOPUP
3bc5214b-0c2d-4c42-a7a4-5e3ce616669a,93,CASH_IN,198237.52,0.0,198237.52,0,SUSPICIOUS_TOPUP
4e701b33-3e38-4f23-89e4-f5e49d0944ee,496,CASH_IN,197983.52,0.0,197983.52,0,SUSPICIOUS_TOPUP
f4522361-211e-4a54-a6e8-e52f2ae0b0a5,157,CASH_IN,197745.15,0.0,197745.15,0,SUSPICIOUS_TOPUP
6396a74c-18b1-4660-9066-6cc4bd5705fb,6,CASH_IN,195788.05,0.0,195788.05,0,SUSPICIOUS_TOPUP
12adec84-3e92-49e6-8567-e06fe06f9f31,8,CASH_IN,195531.5,0.0,195531.5,0,SUSPICIOUS_TOPUP
8e408c00-08e0-4704-850b-cd6958fe392a,496,CASH_IN,195321.24,0.0,195321.24,0,SUSPICIOUS_TOPUP
40f371af-740c-454b-97bb-91873114ef66,93,CASH_IN,195301.71,0.0,195301.71,0,SUSPICIOUS_TOPUP


In [0]:
%sql
SELECT DISTINCT risk_flag
FROM gold_risk_indicators;


risk_flag
SUSPICIOUS_TOPUP
OVERDRAFT
HIGH_AMOUNT
NORMAL


In [0]:
%sql
DESCRIBE TABLE silver_fact_transactions;


col_name,data_type,comment
transaction_id,string,null
step,int,null
transaction_type,string,null
amount,double,null
origin_account,string,null
destination_account,string,null
oldBalanceOrig,double,null
newBalanceOrig,double,null
oldBalanceDest,double,null
newBalanceDest,double,null


In [0]:
%sql
DESCRIBE TABLE silver_dim_fraud_flags;


col_name,data_type,comment
transaction_id,string,null
unauthorized_overdraft_flag,int,null


In [0]:
%sql
CREATE OR REPLACE VIEW workspace.fraud_detection.gold_risk_indicators AS
SELECT
    transaction_id,
    step,
    amount,
    isUnauthorizedOverdraft,
    CASE
        WHEN amount >= 200000 THEN 'HIGH_AMOUNT'
        WHEN isUnauthorizedOverdraft = 1 THEN 'OVERDRAFT'
        WHEN oldBalanceOrig = 0 AND newBalanceOrig > 0 THEN 'SUSPICIOUS_TOPUP'
        ELSE 'NORMAL'
    END AS risk_flag
FROM workspace.fraud_detection.silver_fact_transactions;


In [0]:
%sql
SELECT risk_flag, COUNT(*) 
FROM workspace.fraud_detection.gold_risk_indicators
GROUP BY risk_flag;


risk_flag,COUNT(*)
SUSPICIOUS_TOPUP,526
OVERDRAFT,4454
HIGH_AMOUNT,1085535
NORMAL,4352481


In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA fraud_detection;


In [0]:
%sql
CREATE OR REPLACE VIEW gold_risk_indicators AS
SELECT
    f.transaction_id,
    f.step,
    f.transaction_type,
    f.amount,
    f.origin_account,
    f.destination_account,
    f.isUnauthorizedOverdraft,
    ff.unauthorized_overdraft_flag,

    /* Human-readable risk flag */
    CASE
        WHEN ff.unauthorized_overdraft_flag = 1 THEN 'OVERDRAFT'
        WHEN f.amount >= 200000 THEN 'HIGH_AMOUNT'
        ELSE 'NORMAL'
    END AS risk_flag,

    /* Numeric score for ordering */
    CASE
        WHEN ff.unauthorized_overdraft_flag = 1 THEN 3
        WHEN f.amount >= 200000 THEN 2
        ELSE 1
    END AS risk_score,

    f.ingest_time
FROM silver_fact_transactions f
LEFT JOIN silver_dim_fraud_flags ff
    ON f.transaction_id = ff.transaction_id;


In [0]:
%sql
SELECT risk_flag, COUNT(*)
FROM gold_risk_indicators
GROUP BY risk_flag;


risk_flag,COUNT(*)
OVERDRAFT,12380
HIGH_AMOUNT,1077609
NORMAL,4353007


In [0]:
%sql
SELECT *
FROM gold_risk_indicators
ORDER BY risk_score DESC
LIMIT 10;


transaction_id,step,transaction_type,amount,origin_account,destination_account,isUnauthorizedOverdraft,unauthorized_overdraft_flag,risk_flag,risk_score,ingest_time
0b416cb5-e725-4a05-93bd-eee88b73e923,228,CASH_OUT,131524.28,C4438642533,M1346856382,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
9babc31c-cb9e-41fe-886a-bdac7cc0f939,194,TRANSFER,400000.0,C9743787956,C0054729306,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
3f5dec46-8b7a-4444-bd70-0ecf36f2916d,208,CASH_OUT,133444.54,C5591031120,M7213190977,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
c1c3cc47-1113-4ff5-9b0a-27276466b4e4,357,CASH_OUT,42234.39,C7056550586,M6144123922,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
cf7eb344-f4ab-4345-984c-80810fa931f9,192,CASH_OUT,142565.01,C7733418616,M3228905679,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
cd1c8f6b-e893-49de-a781-6a4fcc14a0ae,285,TRANSFER,400000.0,C4788881119,CC1639315361,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
84de2624-8522-43f0-a5f2-8fef6fa0b238,229,TRANSFER,400000.0,C5357215006,CC9148995407,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
363a7188-550a-49f9-badb-7902b4872055,333,TRANSFER,400000.0,C7586383417,CC7354239311,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
cf3c77f9-42f4-4ca5-a585-321e3f014bff,135,CASH_OUT,189734.16,C5319310142,M1022148075,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
9d8ad561-46df-4f9e-82bd-75722cb9c9fb,10,TRANSFER,400000.0,C1531203291,C6759464356,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z


In [0]:
%sql
DESCRIBE silver_dim_accounts;
DESCRIBE silver_dim_customer;
DESCRIBE silver_fact_transactions;


col_name,data_type,comment
transaction_id,string,null
step,int,null
transaction_type,string,null
amount,double,null
origin_account,string,null
destination_account,string,null
oldBalanceOrig,double,null
newBalanceOrig,double,null
oldBalanceDest,double,null
newBalanceDest,double,null


In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA fraud_detection;


In [0]:
%sql
CREATE OR REPLACE VIEW gold_risk_indicators AS
SELECT
    f.transaction_id,
    f.step,
    f.transaction_type,
    f.amount,
    f.origin_account,
    f.destination_account,
    f.isUnauthorizedOverdraft,

    /* Human-readable risk flag */
    CASE
        WHEN f.isUnauthorizedOverdraft = 1 THEN 'OVERDRAFT'
        WHEN f.amount >= 200000 THEN 'HIGH_AMOUNT'
        ELSE 'NORMAL'
    END AS risk_flag,

    /* Numeric score for ordering */
    CASE
        WHEN f.isUnauthorizedOverdraft = 1 THEN 3
        WHEN f.amount >= 200000 THEN 2
        ELSE 1
    END AS risk_score,

    f.ingest_time
FROM silver_fact_transactions f;


In [0]:
%sql
CREATE OR REPLACE VIEW gold_transactions_summary AS
SELECT
    transaction_type,
    COUNT(*)                 AS total_transactions,
    SUM(amount)              AS total_amount,
    AVG(amount)              AS avg_amount,
    SUM(
        CASE WHEN isUnauthorizedOverdraft = 1 THEN 1 ELSE 0 END
    ) AS overdraft_count
FROM silver_fact_transactions
GROUP BY transaction_type;


In [0]:
%sql
CREATE OR REPLACE VIEW gold_transaction_kpis AS
SELECT
    COUNT(*) AS total_transactions,
    SUM(amount) AS total_amount,
    AVG(amount) AS avg_transaction_amount,
    SUM(
        CASE WHEN isUnauthorizedOverdraft = 1 THEN 1 ELSE 0 END
    ) AS total_overdrafts
FROM silver_fact_transactions;


In [0]:
%sql
SELECT current_catalog(), current_schema();


current_catalog(),current_schema()
workspace,fraud_detection


In [0]:

%sql
SHOW TABLES;


database,tableName,isTemporary
fraud_detection,bronze_country_code,false
fraud_detection,bronze_customers,false
fraud_detection,bronze_fraud_report,false
fraud_detection,bronze_transactions,false
fraud_detection,gold_cross_border_activity,false
fraud_detection,gold_risk_indicators,false
fraud_detection,gold_transaction_kpis,false
fraud_detection,gold_transactions_by_country,false
fraud_detection,gold_transactions_enriched,false
fraud_detection,gold_transactions_summary,false


In [0]:
%sql
DESCRIBE EXTENDED gold_cross_border_activity;


col_name,data_type,comment
origin_country,string,null
destination_country,string,null
transaction_count,bigint,null
total_amount,double,null
avg_amount,double,null
,,
# Detailed Table Information,,
Catalog,workspace,
Database,fraud_detection,
Table,gold_cross_border_activity,


In [0]:
%sql
SELECT * FROM gold_cross_border_activity LIMIT 5;


origin_country,destination_country,transaction_count,total_amount,avg_amount
PER,ESP,5442996,6.502657508176909E11,119468.34993406037
NGA,CAN,5442996,6.502657508176909E11,119468.34993406037
PSE,ITA,5442996,6.502657508176909E11,119468.34993406037
QAT,FRA,5442996,6.502657508176909E11,119468.34993406037
QAT,RUS,5442996,6.502657508176909E11,119468.34993406037


In [0]:
%sql
DESCRIBE EXTENDED gold_cross_border_activity;


col_name,data_type,comment
origin_country,string,null
destination_country,string,null
transaction_count,bigint,null
total_amount,double,null
avg_amount,double,null
,,
# Detailed Table Information,,
Catalog,workspace,
Database,fraud_detection,
Table,gold_cross_border_activity,


In [0]:
%sql
SELECT * FROM gold_cross_border_activity LIMIT 5;


origin_country,destination_country,transaction_count,total_amount,avg_amount
PER,ESP,5442996,6.502657508176909E11,119468.34993406037
NGA,CAN,5442996,6.502657508176909E11,119468.34993406037
PSE,ITA,5442996,6.502657508176909E11,119468.34993406037
QAT,FRA,5442996,6.502657508176909E11,119468.34993406037
QAT,RUS,5442996,6.502657508176909E11,119468.34993406037


In [0]:
%sql
DESCRIBE bronze_transactions;


col_name,data_type,comment
amount,double,null
countryDest,string,null
countryOrig,string,null
customer_id,string,null
id,string,null
isUnauthorizedOverdraft,bigint,null
nameDest,string,null
nameOrig,string,null
newBalanceDest,double,null
newBalanceOrig,double,null


In [0]:
%sql
CREATE OR REPLACE VIEW workspace.fraud_detection.gold_cross_border_activity AS
SELECT
    countryOrig        AS origin_country,
    countryDest        AS destination_country,
    COUNT(*)           AS transaction_count,
    SUM(amount)        AS total_amount,
    AVG(amount)        AS avg_amount
FROM workspace.fraud_detection.bronze_transactions
WHERE countryOrig IS NOT NULL
  AND countryDest IS NOT NULL
  AND countryOrig <> countryDest
GROUP BY countryOrig, countryDest;


In [0]:
%sql
SELECT *
FROM gold_cross_border_activity
ORDER BY transaction_count DESC
LIMIT 10;


origin_country,destination_country,transaction_count,total_amount,avg_amount
NGA,RUS,48981,5.843143376770004E9,119294.08090422826
PSE,RUS,48750,5.856058329520007E9,120124.27342605143
RUS,CAN,48666,5.833012900469998E9,119858.07135310069
RUS,PSE,48651,5.820534787349992E9,119638.54365480652
FRA,RUS,48631,5.795090566849993E9,119164.53634204504
ESP,RUS,48620,5.799697748819996E9,119286.25563183866
RUS,ESP,48586,5.837755970860007E9,120153.047603425
CAN,RUS,48556,5.779333948100011E9,119024.09482041377
PER,RUS,48541,5.827610296060005E9,120055.42316927969
RUS,REU,48519,5.773528334420004E9,118995.20465013714


In [0]:
%sql
CREATE OR REPLACE VIEW workspace.fraud_detection.gold_risk_indicators AS
SELECT
    f.transaction_id,
    f.step,
    f.transaction_type,
    f.amount,
    f.origin_account,
    f.destination_account,
    f.isUnauthorizedOverdraft,

    ff.unauthorized_overdraft_flag,

    /* Human-readable risk label */
    CASE
        WHEN ff.unauthorized_overdraft_flag = 1 THEN 'OVERDRAFT'
        WHEN f.amount >= 200000 THEN 'HIGH_AMOUNT'
        ELSE 'NORMAL'
    END AS risk_flag,

    /* Numeric severity for ordering */
    CASE
        WHEN ff.unauthorized_overdraft_flag = 1 THEN 3
        WHEN f.amount >= 200000 THEN 2
        ELSE 1
    END AS risk_score,

    f.ingest_time
FROM workspace.fraud_detection.silver_fact_transactions f
LEFT JOIN workspace.fraud_detection.silver_dim_fraud_flags ff
    ON f.transaction_id = ff.transaction_id;


In [0]:
%sql
SELECT risk_flag, COUNT(*) 
FROM gold_risk_indicators
GROUP BY risk_flag;


risk_flag,COUNT(*)
OVERDRAFT,12380
HIGH_AMOUNT,1077609
NORMAL,4353007


In [0]:
%sql
SELECT *
FROM gold_risk_indicators
ORDER BY risk_score DESC, amount DESC
LIMIT 20;


transaction_id,step,transaction_type,amount,origin_account,destination_account,isUnauthorizedOverdraft,unauthorized_overdraft_flag,risk_flag,risk_score,ingest_time
b8b91c31-9587-4cc1-94ed-9061f75c28da,408,CASH_OUT,406958.26,C4538577357,M7644246411,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
b6e635be-7ff1-4e7e-90e0-7f2a3f8783d8,302,TRANSFER,400000.0,C8706986180,CC3109056341,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
40783acc-8761-4b46-8b6c-5f52993f3cf3,356,TRANSFER,400000.0,C8752163755,CC0772297961,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
9babc31c-cb9e-41fe-886a-bdac7cc0f939,194,TRANSFER,400000.0,C9743787956,C0054729306,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
9d8ad561-46df-4f9e-82bd-75722cb9c9fb,10,TRANSFER,400000.0,C1531203291,C6759464356,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
59b80037-fe2a-4ac9-932f-d4e611c7a8dc,313,TRANSFER,400000.0,C9293144756,CC8496270202,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
84de2624-8522-43f0-a5f2-8fef6fa0b238,229,TRANSFER,400000.0,C5357215006,CC9148995407,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
cd1c8f6b-e893-49de-a781-6a4fcc14a0ae,285,TRANSFER,400000.0,C4788881119,CC1639315361,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
6aa2c249-3f14-46cd-9bf2-b143e58d21f0,212,TRANSFER,400000.0,C6614089555,C8302914178,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z
f980f493-4d84-490c-a2b8-6186c218113c,119,TRANSFER,400000.0,C5102874688,CC8201991534,1,1,OVERDRAFT,3,2026-02-08T21:37:25.820Z


In [0]:
%sql
CREATE OR REPLACE VIEW workspace.fraud_detection.gold_transaction_kpis AS
SELECT
    COUNT(*)                              AS total_transactions,
    SUM(amount)                           AS total_amount,
    AVG(amount)                           AS avg_amount,

    SUM(CASE WHEN risk_flag = 'HIGH_AMOUNT' THEN 1 ELSE 0 END) 
        AS high_amount_txn_count,

    SUM(CASE WHEN risk_flag = 'OVERDRAFT' THEN 1 ELSE 0 END) 
        AS overdraft_txn_count,

    ROUND(
        100.0 * SUM(CASE WHEN risk_flag = 'OVERDRAFT' THEN 1 ELSE 0 END) 
        / COUNT(*),
        2
    ) AS overdraft_percentage
FROM gold_risk_indicators;


In [0]:
%sql
SELECT * FROM gold_transaction_kpis;


total_transactions,total_amount,avg_amount,high_amount_txn_count,overdraft_txn_count,overdraft_percentage
5442996,6.502657508176656E11,119468.34993405573,1077609,12380,0.23


In [0]:
%sql
CREATE OR REPLACE VIEW workspace.fraud_detection.gold_transactions_summary AS
SELECT
  step,

  COUNT(*)                                   AS txn_count,
  SUM(amount)                                AS total_amount,
  AVG(amount)                                AS avg_amount,

  SUM(CASE WHEN risk_flag = 'OVERDRAFT'   THEN 1 ELSE 0 END) AS overdraft_txn_count,
  SUM(CASE WHEN risk_flag = 'HIGH_AMOUNT' THEN 1 ELSE 0 END) AS high_amount_txn_count,
  SUM(CASE WHEN risk_flag = 'NORMAL'      THEN 1 ELSE 0 END) AS normal_txn_count,

  ROUND(100.0 * SUM(CASE WHEN risk_flag = 'OVERDRAFT' THEN 1 ELSE 0 END) / COUNT(*), 2) AS overdraft_pct,
  ROUND(100.0 * SUM(CASE WHEN risk_flag = 'HIGH_AMOUNT' THEN 1 ELSE 0 END) / COUNT(*), 2) AS high_amount_pct

FROM workspace.fraud_detection.gold_risk_indicators
GROUP BY step
ORDER BY step;


In [0]:
%sql
SELECT * 
FROM workspace.fraud_detection.gold_transactions_summary
ORDER BY step
LIMIT 20;


step,txn_count,total_amount,avg_amount,overdraft_txn_count,high_amount_txn_count,normal_txn_count,overdraft_pct,high_amount_pct
0,102,2966633.6700000004,29084.643823529415,0,4,98,0.00,3.92
1,16495,2.1335749066000004E9,129346.76608669297,151,3029,13315,0.92,18.36
2,16515,2.2053613694700003E9,133536.8676639419,82,3280,13153,0.50,19.86
3,16675,2.1975923971700006E9,131789.6490056972,17,3062,13596,0.10,18.36
4,362,1.2040243047E8,332603.3990883978,0,301,61,0.00,83.15
5,304,1.0403221063E8,342211.21917763154,0,266,38,0.00,87.50
6,16704,2.219528043769999E9,132874.04476592428,41,3510,13153,0.25,21.01
7,17074,2.2359628636099987E9,130957.17837706447,54,3341,13679,0.32,19.57
8,17073,2.2511718293800006E9,131855.66856322854,51,3516,13506,0.30,20.59
9,17279,2.3166803343699994E9,134074.90794432544,55,3807,13417,0.32,22.03


In [0]:
%sql
SELECT MIN(step) AS min_step, MAX(step) AS max_step, COUNT(*) AS step_rows
FROM workspace.fraud_detection.gold_transactions_summary;


min_step,max_step,step_rows
0,601,602


In [0]:
%sql
SELECT *
FROM workspace.fraud_detection.gold_transactions_summary
WHERE step >= (SELECT MAX(step) - 50 FROM workspace.fraud_detection.gold_transactions_summary)
ORDER BY step;


step,txn_count,total_amount,avg_amount,overdraft_txn_count,high_amount_txn_count,normal_txn_count,overdraft_pct,high_amount_pct
551,16646,1.0751366428799994E9,64588.288049981944,14,1462,15170,0.08,8.78
552,16695,1.0845583808200002E9,64963.06563761606,12,1983,14700,0.07,11.88
553,197,6.505359249999999E7,330221.2817258883,0,158,39,0.00,80.20
554,273,9.281158392E7,339969.1718681319,0,233,40,0.00,85.35
555,16650,5.002857878100002E8,30047.194463063075,2,342,16306,0.01,2.05
556,242,8.111939895E7,335204.127892562,0,205,37,0.00,84.71
557,355,1.2286968396000002E8,346111.785802817,0,310,45,0.00,87.32
558,206,6.634830938E7,322079.17174757284,0,169,37,0.00,82.04
559,247,8.528942011E7,345301.2959919028,0,216,31,0.00,87.45
560,234,7.820891213E7,334226.1202136752,0,195,39,0.00,83.33


In [0]:
%sql
SELECT
  step,
  overdraft_txn_count,
  high_amount_txn_count,
  normal_txn_count
FROM workspace.fraud_detection.gold_transactions_summary
ORDER BY step;


step,overdraft_txn_count,high_amount_txn_count,normal_txn_count
0,0,4,98
1,151,3029,13315
2,82,3280,13153
3,17,3062,13596
4,0,301,61
5,0,266,38
6,41,3510,13153
7,54,3341,13679
8,51,3516,13506
9,55,3807,13417


In [0]:
%sql
CREATE OR REPLACE VIEW workspace.fraud_detection.gold_transaction_kpis AS
SELECT
    COUNT(*)        AS total_transactions,
    SUM(amount)     AS total_amount,
    AVG(amount)     AS avg_amount,

    SUM(CASE WHEN risk_flag = 'HIGH_AMOUNT' THEN 1 ELSE 0 END)
        AS high_amount_txn_count,

    SUM(CASE WHEN risk_flag = 'OVERDRAFT' THEN 1 ELSE 0 END)
        AS overdraft_txn_count,

    ROUND(
        100.0 * SUM(CASE WHEN risk_flag = 'OVERDRAFT' THEN 1 ELSE 0 END)
        / COUNT(*),
        2
    ) AS overdraft_percentage

FROM workspace.fraud_detection.gold_risk_indicators;


In [0]:
%sql
SELECT *
FROM workspace.fraud_detection.gold_transactions_summary
ORDER BY step
LIMIT 20;


step,txn_count,total_amount,avg_amount,overdraft_txn_count,high_amount_txn_count,normal_txn_count,overdraft_pct,high_amount_pct
0,102,2966633.6700000004,29084.643823529415,0,4,98,0.00,3.92
1,16495,2.1335749066000004E9,129346.76608669297,151,3029,13315,0.92,18.36
2,16515,2.2053613694700003E9,133536.8676639419,82,3280,13153,0.50,19.86
3,16675,2.1975923971700006E9,131789.6490056972,17,3062,13596,0.10,18.36
4,362,1.2040243047E8,332603.3990883978,0,301,61,0.00,83.15
5,304,1.0403221063E8,342211.21917763154,0,266,38,0.00,87.50
6,16704,2.219528043769999E9,132874.04476592428,41,3510,13153,0.25,21.01
7,17074,2.2359628636099987E9,130957.17837706447,54,3341,13679,0.32,19.57
8,17073,2.2511718293800006E9,131855.66856322854,51,3516,13506,0.30,20.59
9,17279,2.3166803343699994E9,134074.90794432544,55,3807,13417,0.32,22.03


In [0]:
%sql
SELECT
  MIN(step) AS min_step,
  MAX(step) AS max_step,
  COUNT(*)  AS steps
FROM workspace.fraud_detection.gold_transactions_summary;


min_step,max_step,steps
0,601,602


In [0]:
%sql
SELECT * 
FROM workspace.fraud_detection.gold_transactions_summary
ORDER BY step
LIMIT 20;


step,txn_count,total_amount,avg_amount,overdraft_txn_count,high_amount_txn_count,normal_txn_count,overdraft_pct,high_amount_pct
0,102,2966633.6700000004,29084.643823529415,0,4,98,0.00,3.92
1,16495,2.1335749066000004E9,129346.76608669297,151,3029,13315,0.92,18.36
2,16515,2.2053613694700003E9,133536.8676639419,82,3280,13153,0.50,19.86
3,16675,2.1975923971700006E9,131789.6490056972,17,3062,13596,0.10,18.36
4,362,1.2040243047E8,332603.3990883978,0,301,61,0.00,83.15
5,304,1.0403221063E8,342211.21917763154,0,266,38,0.00,87.50
6,16704,2.219528043769999E9,132874.04476592428,41,3510,13153,0.25,21.01
7,17074,2.2359628636099987E9,130957.17837706447,54,3341,13679,0.32,19.57
8,17073,2.2511718293800006E9,131855.66856322854,51,3516,13506,0.30,20.59
9,17279,2.3166803343699994E9,134074.90794432544,55,3807,13417,0.32,22.03
